In [1]:


import torch
import numpy as np
import sys
import os 
import torch.nn as nn
import torch.nn.functional as F
import yaml
import pickle
from model import AE
from utils import *
from functools import reduce
import json
from collections import defaultdict
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from argparse import ArgumentParser, Namespace
from scipy.io.wavfile import write
import random
from preprocess.tacotron.utils import melspectrogram2wav
from preprocess.tacotron.utils import get_spectrograms
import librosa 

class Inferencer(object):
    def __init__(self, config, args):
        # config store the value of hyperparameters, turn to attr by AttrDict
        self.config = config
        print(config)
        # args store other information
        self.args = args
        print(self.args)

        # init the model with config
        self.build_model()

        # load model
        self.load_model()

        with open(self.args.attr, 'rb') as f:
            self.attr = pickle.load(f)

    def load_model(self):
        print(f'Load model from {self.args.model}')
        self.model.load_state_dict(torch.load(f'{self.args.model}',map_location="cpu"))
        return

    def build_model(self): 
        # create model, discriminator, optimizers
        #self.model = cc(AE(self.config))
        self.model=  AE(self.config).to(torch.device('cpu'))
        print(self.model)
        self.model.eval()
        return

    def utt_make_frames(self, x):
        frame_size = self.config['data_loader']['frame_size']
        remains = x.size(0) % frame_size 
        if remains != 0:
            x = F.pad(x, (0, remains))
        out = x.view(1, x.size(0) // frame_size, frame_size * x.size(1)).transpose(1, 2)
        return out

    def inference_one_utterance(self, x, x_cond):
        x = self.utt_make_frames(x)
        x_cond = self.utt_make_frames(x_cond)
        dec = self.model.inference(x, x_cond)
        dec = dec.transpose(1, 2).squeeze(0)
        dec = dec.detach().cpu().numpy()
        dec = self.denormalize(dec)
        wav_data = melspectrogram2wav(dec)
        return wav_data, dec

    def denormalize(self, x):
        m, s = self.attr['mean'], self.attr['std']
        ret = x * s + m
        return ret

    def normalize(self, x):
        m, s = self.attr['mean'], self.attr['std']
        ret = (x - m) / s
        return ret

    def write_wav_to_file(self, wav_data, output_path):
        write(output_path, rate=self.args.sample_rate, data=wav_data)
        return

    def inference_from_path(self):
        src_mel, _ = get_spectrograms(self.args.source)
        tar_mel, _ = get_spectrograms(self.args.target)
        src_mel = torch.from_numpy(self.normalize(src_mel))
        tar_mel = torch.from_numpy(self.normalize(tar_mel))
        conv_wav, conv_mel = self.inference_one_utterance(src_mel, tar_mel)
        self.write_wav_to_file(conv_wav, self.args.output)
        return



In [2]:
parser = ArgumentParser()
parser.add_argument('-attr', '-a', help='attr file path')
parser.add_argument('-config', '-c', help='config file path')
parser.add_argument('-model', '-m', help='model path')
parser.add_argument('-source', '-s', help='source wav path')
parser.add_argument('-target', '-t', help='target wav path')
parser.add_argument('-output', '-o', help='output wav path')
parser.add_argument('-sample_rate', '-sr', help='sample rate', default=24000, type=int)
args = parser.parse_args(("-a spectrograms/sr_24000_mel_norm_128frame_256mel/attr.pkl -c ../models/setup2.config.yaml -m ../models/setup2.ckpt" + 
                          " -s ../../ArcticL2/NCC/wav/arctic_a0001.wav -o here.wav -t ../../L1Arctic/cmu_us_slt_arctic/wav/arctic_a0001.wav").split())
# load config file 
with open(args.config) as f:
    config = yaml.load(f)
inferencer = Inferencer(config=config, args=args)
inferencer.inference_from_path()


{'ContentEncoder': {'act': 'relu', 'bank_scale': 1, 'bank_size': 8, 'c_bank': 128, 'c_h': 128, 'c_in': 256, 'c_out': 128, 'dropout_rate': 0, 'kernel_size': 5, 'n_conv_blocks': 6, 'subsample': [1, 2, 1, 2, 1, 2]}, 'Decoder': {'act': 'relu', 'c_cond': 128, 'c_h': 128, 'c_in': 128, 'c_out': 256, 'dropout_rate': 0, 'kernel_size': 5, 'n_conv_blocks': 6, 'sn': False, 'upsample': [2, 1, 2, 1, 2, 1]}, 'SpeakerEncoder': {'act': 'relu', 'bank_scale': 1, 'bank_size': 8, 'c_bank': 128, 'c_h': 128, 'c_in': 256, 'c_out': 128, 'dropout_rate': 0, 'kernel_size': 5, 'n_conv_blocks': 6, 'n_dense_blocks': 6, 'subsample': [1, 2, 1, 2, 1, 2]}, 'annealing_iters': 20000, 'data_loader': {'batch_size': 128, 'frame_size': 1, 'segment_size': 128, 'shuffle': True}, 'lambda': {'lambda_kl': 1, 'lambda_rec': 10}, 'optimizer': {'amsgrad': True, 'beta1': 0.9, 'beta2': 0.999, 'grad_norm': 5, 'lr': 0.0005, 'weight_decay': 0.0001}}
Namespace(attr='spectrograms/sr_24000_mel_norm_128frame_256mel/attr.pkl', config='../models

In [3]:
from IPython.display import Audio

In [4]:
Audio("here.wav")

In [5]:
Audio("../../ArcticL2/NCC/wav/arctic_a0001.wav")

In [6]:
Audio("../../L1Arctic/cmu_us_slt_arctic/wav/arctic_a0001.wav")

In [ ]:
Testing ids:  ['slt', 'YBAA', 'NCC', 'RRBI', 'YDCK', 'ERMS', 'THV']
